In [1]:
# Drive setup
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm.auto import tqdm
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.utils.data import sampler
from torchvision import datasets
import torch.nn.functional as F
import torchvision.datasets as dset
import torchvision.transforms as T
from google.colab.patches import cv2_imshow
from torchvision.transforms import ToTensor
 
import matplotlib.pyplot as plt
from google.colab.patches import cv2_imshow
import cv2
import pickle
import os
import pandas as pd
import random
import numpy as np
from torchvision.io import ImageReadMode, read_image
from torch.utils.data import DataLoader
import matplotlib.image as mpimg
import cv2
import pickle
import warnings
import copy
import pickle
from google.colab.patches import cv2_imshow
import random
from sklearn.metrics import average_precision_score
import math
import torchvision
warnings.filterwarnings('ignore')
from torch import tensor
from torchvision.ops.boxes import box_iou

## Data Preparation


In [3]:
with open('/content/drive/MyDrive/SSD detection/images_2.pkl', 'rb') as f:
    images = pickle.load(f)
with open('/content/drive/MyDrive/SSD detection/labels_2.pkl', 'rb') as f:
    labels = pickle.load(f)
labels = [[label] for label in labels]
with open('/content/drive/MyDrive/SSD detection/coco_person_fire_hydrant_image_3.pickle', 'rb') as f:
    images_rest = pickle.load(f)
    # images_rest = []
for idx in range(len(images_rest)):
  images_rest[idx] = np.array(images_rest[idx])
with open('/content/drive/MyDrive/SSD detection/coco_person_fire_hydrant_class_labels_3.pickle', 'rb') as f:
    labels_rest = pickle.load(f)
    # labels_rest = []
with open('/content/drive/MyDrive/SSD detection/coco_person_fire_hydrant_annotation_3.pickle', 'rb') as f:
    annotations = pickle.load(f)

In [4]:
random.Random(4).shuffle(images)
random.Random(4).shuffle(labels)
random.Random(5).shuffle(images_rest)
random.Random(5).shuffle(labels_rest)
random.Random(5).shuffle(annotations)

In [5]:
print(len(images))
print(len(labels))
print(len(images_rest))
print(len(labels_rest))
print(len(annotations))
# print(len(images_door))
# print(len(annotations_door))

459
459
438
438
438


In [6]:
print(labels[:5])
print(labels_rest[:5])
print(annotations[:5])
# print(annotations_door[:5])

[[[137, 91, 226, 217]], [[96, 109, 123, 142]], [[88, 176, 163, 281]], [[67, 1, 163, 138]], [[79, 2, 246, 260]]]
[['fire hydrant', 'person', 'person'], ['person', 'person', 'fire hydrant'], ['person', 'person', 'fire hydrant'], ['person', 'person', 'fire hydrant', 'person'], ['fire hydrant', 'person', 'person', 'person']]
[[[104, 123, 237, 313], [32, 19, 156, 283], [184, 34, 295, 276]], [[33, 47, 178, 305], [176, 62, 303, 315], [155, 52, 236, 319]], [[234, 24, 303, 278], [115, 50, 145, 85], [15, 243, 76, 319]], [[85, 165, 98, 188], [66, 167, 73, 188], [190, 210, 214, 226], [8, 168, 14, 176]], [[78, 48, 204, 249], [55, 0, 120, 81], [197, 0, 301, 53], [96, 0, 155, 66]]]


Processing images in COCO - Making sure that very small humans are not taken into consideration in the model (harm the results a lot).

In [7]:
images_coco = []
labels_coco = []
annotations_coco = []
val = 0
for img in images_rest:
    cnt = 0
    label_here = []
    annotation_here = []
    z = 0
    if len(annotations[val]) > 4:
      val = val + 1
      continue
    for label in annotations[val]:
      # try:
        # Checking width and person class
        height, width = img.shape[:2]
        if ((label[2]-label[0])*(label[3]-label[1]))/(width*height) < 0.005:
            continue
      # except:
        # print(label)
        annotation_here.append(label)
        label_here.append(labels_rest[val][z])
        cnt += 1
        z += 1
    val += 1
    if cnt > 0:
        images_coco.append(img)
        labels_coco.append(label_here)
        annotations_coco.append(annotation_here)

# print(len(images_coco))

Removing weird images in the dataset (Black & White/malformed).

In [8]:
def remove_wrong_images(images, labels, annotations = None):
    images_new = []
    labels_new = []
    annotations_new = []
    cnt = 0
    for cnt in range(len(images)):
        # Checking malformed images
        if len(images[cnt].shape) < 3 or images[cnt].shape[2] != 3:
            continue
        images_new.append(images[cnt])
        labels_new.append(labels[cnt])
        if annotations is not None:
            annotations_new.append(annotations[cnt])
    return images_new, labels_new, annotations_new

images_coco, annotations_coco, labels_coco = remove_wrong_images(images_coco, annotations_coco, labels_coco)
# images_door, annotations_door, _ = remove_wrong_images(images_door, annotations_door)
images, labels, _ = remove_wrong_images(images, labels)

In [9]:
def image_augmentation_flip_horizontal(images):
  original_images = images
  new_images = []
  for idx in range(len(new_images)):
    image = cv2.flip(new_images[idx], 1)#*255
    new_images.append(image)
    # cv2_imshow(image)
  original_images.extend(new_images)
  return original_images

In [10]:
import copy
def image_augmentation_translation(images_list, labels_list, annotations_list):
  original_images_list = copy.deepcopy(images_list)
  original_labels_list = copy.deepcopy(labels_list)
  original_annotations_list = copy.deepcopy(annotations_list)
  new_images_list = []
  new_labels_list = []
  new_annotations_list = []
  for img, labels, annotations in zip(images_list, labels_list, annotations_list):
    # annotations = [annotations]
    height, width = img.shape[:2]
    translation_factor = 0.2
    max_horizontal = translation_factor*width
    max_vertical = translation_factor*height
    horizontal_shift = int(random.uniform(-max_horizontal, max_horizontal))
    vertical_shift = int(random.uniform(-max_vertical, max_vertical))
    # M[0][2] = how much right
    # M[1][2] = how much down
    M = np.float32([[1, 0, horizontal_shift], [0, 1, vertical_shift]])
    new_img = cv2.warpAffine(img, M, (width, height))
    # filling blank spaces in image
    if horizontal_shift >= 0:
      new_img[:,:horizontal_shift] = np.random.rand(320, horizontal_shift, 3)
    else:
     new_img[:,horizontal_shift:] = np.random.rand(320, -horizontal_shift, 3)
    if vertical_shift >= 0:
      new_img[:vertical_shift,:] = np.random.rand(vertical_shift, 320,3)
    else:
      new_img[vertical_shift:,:] = np.random.rand(-vertical_shift, 320,3)

    # new_images.append(new_img)
    temp_label = []
    temp_annotations = []
    for label, box in zip(labels, annotations):
      x1 = box[0]
      y1 = box[1]
      x2 = box[2]
      y2 = box[3]
      x1 = x1 + horizontal_shift
      y1 = y1 + vertical_shift
      x2 = x2 + horizontal_shift
      y2 = y2 + vertical_shift
      if x1 >= 0 and x1 < width and y1 >= 0 and y1 < height and x2 >= 0 and x2 < width and y2 >= 0 and y2 < height:
        temp_label.append(label) # No cutting
        temp_annotations.append([x1, y1, x2, y2]) # No cutting just translation
      elif x1 >= width or y1 >= height or x2 < 0 or y2 < 0:
        pass # whole out
      else: # for cases in which there is partial image cut [if >=30% is image has gone out of bounds then remove else make modifications and add]
        # find new coordinates of bbox
        initial_area = (x2-x1)*(y2-y1)
        if x1 < 0:
          x1 = 0
        if y1 < 0:
          y1 = 0
        if x2 >= width:
          x2 = width - 1
        if y2 >= height:
          y2 = height - 1
        final_area = (x2-x1)*(y2-y1)
        # check if more than 50 % has gone out
        if final_area >= 0.7*initial_area:
          # accept modified
          temp_annotations.append([x1, y1, x2, y2])
          temp_label.append(label)
        else:
          pass
        # do something 
    if len(temp_annotations) > 0:
      new_images_list.append(new_img)
      new_labels_list.append(temp_label)
      new_annotations_list.append(temp_annotations)

    # print(horizontal_shift, vertical_shift)
    # cv2_imshow(translated)
  result_images_list = []
  result_labels_list = []
  result_annotations_list = []
  for img1, img2 in zip(original_images_list, new_images_list):
    result_images_list.append(img1)
    result_images_list.append(img2)
  for label1, label2 in zip(original_labels_list, new_labels_list):
    result_labels_list.append(label1)
    result_labels_list.append(label2)
  for annotation1, annotation2 in zip(original_annotations_list, new_annotations_list):
    result_annotations_list.append(annotation1)
    result_annotations_list.append(annotation2)

  return result_images_list, result_labels_list, result_annotations_list

index1 = random.randint(0, len(images_coco))
index2 = random.randint(0, len(images_coco))
index3 = random.randint(0, len(images_coco))
dummy_img_list = [images_coco[index1], images_coco[index2], images_coco[index3]]
dummy_class_list = [labels_coco[index1], labels_coco[index2], labels_coco[index3]]
dummy_annotations_list = [annotations_coco[index1], annotations_coco[index2], annotations_coco[index3]] 

# dummy_img_list, dummy_class_list, dummy_annotations_list = image_augmentation_translation(dummy_img_list, dummy_class_list, dummy_annotations_list)
# images, dummy_class_list, labels = image_augmentation_translation(images, [[1]]*len(images), labels)
# images_coco, labels_coco, annotations_coco = image_augmentation_translation(images_coco, labels_coco, annotations_coco)

In [11]:
import copy
def img_list_viewer(images_list, labels_list, annotations_list):
  for img, labels, annotations in zip(images_list, labels_list, annotations_list):
    temp_img = copy.deepcopy(img)
    # print(labels, annotations)
    for label, annotation in zip(labels, annotations):
      # print(label, annotation)
      color = None
      if label == "doll":
        color = (1, 0, 0)
      elif label == "fire hydrant":
        color = (0, 1 ,0)
      elif label == "person":
        color = (0, 0 ,1)
      # print(annotations)
      temp_img = cv2.rectangle(temp_img, (annotation[0], annotation[1]), (annotation[2], annotation[3]), color, 2)
    temp_img = temp_img*255
    temp_img = np.array(temp_img, dtype=np.uint8)
    temp_img = cv2.cvtColor(temp_img, cv2.COLOR_RGB2BGR)
    cv2_imshow(temp_img)
  return

In [ ]:
dummy_img, dummy_label, dummy_annotation = image_augmentation_translation(images[:5], [["doll"]]*len(labels[:5]), labels[:5])
img_list_viewer(dummy_img, dummy_label, dummy_annotation)

In [13]:
# counting no of samples per class
print("No. of doll samples : ", len(images))
count_person = 0
count_fire_hydrant = 0
flat_labels_coco = [item for sublist in labels_coco for item in sublist]
for class_name in flat_labels_coco:
  if class_name == "person":
    count_person = count_person + 1
  elif class_name == "fire hydrant":
    count_fire_hydrant = count_fire_hydrant + 1
# print(flat_labels_coco[:5])
print("No of person samples : ", count_person)
print("No of fire hydrant samples : ", count_fire_hydrant)

No. of doll samples :  459
No of person samples :  481
No of fire hydrant samples :  305


In [ ]:
dummy_img, dummy_label, dummy_annotation = image_augmentation_translation(images_rest[:5], labels_rest[:5], annotations[:5])
img_list_viewer(dummy_img, dummy_label, dummy_annotation)

In [15]:
images, dummy_list, labels = image_augmentation_translation(images, [["doll"]]*len(labels), labels)
images_coco, labels_coco, annotations_coco= image_augmentation_translation(images_coco, labels_coco, annotations_coco)

In [16]:
# images, dummy_list, labels = image_augmentation_translation(images, [["doll"]]*len(labels), labels)
# images_coco, labels_coco, annotations_coco= image_augmentation_translation(images_coco, labels_coco, annotations_coco)

In [17]:
# counting no of samples per class
print("No. of doll samples : ", len(images))
count_person = 0
count_fire_hydrant = 0
flat_labels_coco = [item for sublist in labels_coco for item in sublist]
for class_name in flat_labels_coco:
  if class_name == "person":
    count_person = count_person + 1
  elif class_name == "fire hydrant":
    count_fire_hydrant = count_fire_hydrant + 1
# print(flat_labels_coco[:5])
print("No of person samples : ", count_person)
print("No of fire hydrant samples : ", count_fire_hydrant)

No. of doll samples :  874
No of person samples :  885
No of fire hydrant samples :  570


## Data Augmentation
The following image transform classes were defined to deal with bounding boxes in transformations.
1. Random Crop (resized to full)
2. Perspective Transformation

In [18]:
# For reference, see PyTorch's implementation of T.RandomResizedCrop
class RandomResizedCropWithBox(T.RandomResizedCrop):
    def __init__(self, *args, **kwargs):
        super(RandomResizedCropWithBox, self).__init__(*args, **kwargs)

    def forward(self, img_data):
        img = img_data[0]
        boxes = img_data[1]
        classes = img_data[2]
        i, j, h, w = self.get_params(img, self.scale, self.ratio)
        num_boxes = len(boxes)
        new_labels = []
        new_classes = []

        # Creating labels for each bounding box
        for val in range(num_boxes):
            xmin, ymin, xmax, ymax = boxes[val]
            # Checking if it lies inside the cropped image
            if xmax <= j or xmin >= j+w or ymax <= i or ymin >= i+h:
                continue
            x1 = (max(xmin, j)-j)*320/w
            x2 = (min(xmax, j+w)-j)*320/w
            y1 = (max(ymin, i)-i)*320/h
            y2 = (min(ymax, i+h)-i)*320/h
            for value in (x1, x2, y1, y2):
                if value < 0:
                    value = 0
                if value >= 320:
                    value = 319
            new_labels.append([x1, y1, x2, y2])
            new_classes.append(classes[val])
        new_labels = torch.from_numpy(np.array(new_labels)).int()

        # Returns resized image, labels (box co-ordinates) and classes
        return [torchvision.transforms.functional.resized_crop(img, i, j, h, w, self.size, self.interpolation), new_labels, new_classes]

The custom dataset class which outputs objects according to `idx` - Different ranges give objects from different arrays above.

`collate_fn` is used to get the correct format of values from the dataloader (to handle the dictionaries correctly).

In [19]:
class DollDataset(Dataset):

    # All arrays and values that are part of the Dataset class
    def __init__(self, images, labels, images_coco, labels_coco, annotations, xsize, ysize, perspective_prob = 0.5):
        self.images = images
        self.labels = labels
        self.images_rest = images_coco
        self.labels_rest = labels_coco
        self.annotations = annotations
        # self.images_doors = images_doors
        # self.labels_doors = labels_doors
        self.xsize = xsize
        self.ysize = ysize
        self.perspective_prob = perspective_prob
    
    # Combining 3 arrays
    def __len__(self):
        return len(self.images)+len(self.images_rest)#+len(self.images_doors)

    # Crucial function, returns (non-)transformed image with box
    def __getitem__(self, idx):
        
        # Normalize with calculated mean and std dev
        trans = T.Compose([T.ToTensor(), T.Resize((self.xsize,self.ysize)), T.Normalize([0.4662, 0.4279, 0.3946], [0.2736, 0.2650, 0.2774])])
        # trans = T.Compose([T.ToTensor(), T.Resize((self.xsize,self.ysize))])

        # Dealing with individual arrays
        z = len(self.images)
        z2 = len(self.images_rest)
        # print(z, z+z2)
        if idx < z:
            class_list = [1]
        elif idx < z+z2:
            class_list = self.labels_rest[idx-z]
            for i in range(len(class_list)):
                if class_list[i] == 'person':
                    class_list[i] = 3
                if class_list[i] == 'fire hydrant':
                    class_list[i] = 2
        # else:
        #     class_list = [3]*len(self.labels_doors[idx-z-z2])
        try:
          if idx < z:
              img = self.images[idx]
          elif idx < z+z2:
              img = self.images_rest[idx-z]
        except:
          print("from img ", idx)
        # else:
        #     img = self.images_doors[idx-z-z2] 
        try:
          if idx < z:
              label = np.array(self.labels[idx])
          elif idx < z+z2:
              label = np.array(self.annotations[idx-z])
          else:
            print("from label ", idx)
        except:
          print("from label ", idx)
        # else:
        #     label = np.array(self.labels_doors[idx-z-z2])

        # Label resizing
        label = torch.from_numpy(label)
        y_size, x_size = img.shape[:2]
        label[:,1] = label[:,1]*320/y_size
        label[:,3] = label[:,3]*320/y_size
        label[:,0] = label[:,0]*320/x_size
        label[:,2] = label[:,2]*320/x_size
        label = label.int()
        # print(img)
        img = trans(img)
        return (img, label, class_list)

# Necessary to form a dataloader
def collate_fn(data):
    dics = []
    for x in range(len(data)):
        dic = {'image': data[x][0], 'bbox': data[x][1], 'label': torch.tensor(data[x][2])}
        dics.append(dic)
    return dics

print(len(images)*4//5, len(images_coco)*4//5)

a = len(images)*4//5
b = len(images_coco)*4//5


# train dataset
random.Random(6).shuffle(images[:a])
random.Random(6).shuffle(labels[:a])
random.Random(6).shuffle(images_coco[:b])
random.Random(6).shuffle(labels_coco[:b])
random.Random(6).shuffle(annotations_coco[:b])
train_set = DollDataset(images[:a], labels[:a], images_coco[:b], labels_coco[:b], annotations_coco[:b], 320, 320, perspective_prob=-1)

print(len(images[a:]), len(images_coco[b:]))
# test dataset
random.Random(2).shuffle(images[a:])
random.Random(2).shuffle(labels[a:])
random.Random(2).shuffle(images_coco[b:])
random.Random(2).shuffle(labels_coco[b:])
random.Random(2).shuffle(annotations_coco[b:])
val_set = DollDataset(images[a:], labels[a:], images_coco[b:], labels_coco[b:], annotations_coco[b:], 320, 320, perspective_prob=-1)

# print(train_set.__len__())
# print(val_set.__len__())

# Training and Validation
# train_set, _ = torch.utils.data.random_split(doll_set1, [doll_set1.__len__()*1, 0])
# val_set, _ = torch.utils.data.random_split(doll_set2, [doll_set2.__len__()*1, 0])
train_loader = DataLoader(train_set, batch_size = 16, shuffle = True, collate_fn = collate_fn, drop_last = True)
val_loader = DataLoader(val_set, batch_size = 16, shuffle = True, collate_fn = collate_fn, drop_last = True)

699 595
175 149


In [20]:
# print(doll_set[0][0])

In [21]:
# print(doll_set[7][0])

In [22]:
print(len(train_set), len(val_set))

1294 324


In [23]:
# train_set[2430][1]

In [24]:
# Testing if the dataloader works
for dic in tqdm(train_loader):
    break

  0%|          | 0/80 [00:00<?, ?it/s]

Testing the working of the dataloader using the ```show``` function for torch tensors.

In [25]:
# a = 

In [26]:
# import torchvision.transforms.functional as F
# import random

# random.seed(0)
# torch.manual_seed(0)
# np.random.seed(0)

# # Custom function to display images
# def show(imgs):
#     if not isinstance(imgs, list):
#         imgs = [imgs]
#     fix, axs = plt.subplots(ncols=len(imgs), squeeze=False)
#     for i, img in enumerate(imgs):
#         img = img.detach()
#         img = F.to_pil_image(img)
#         axs[0, i].imshow(np.asarray(img))
#         axs[0, i].set(xticklabels=[], yticklabels=[], xticks=[], yticks=[])

# from torchvision.transforms.functional import convert_image_dtype
# from torchvision.utils import draw_bounding_boxes
# plt.rcParams["savefig.bbox"] = 'tight'
# # T.Normalize([70.0594, 62.4050, 58.8377], [78.0825, 72.5514, 73.4684]
# # Showing one batch of images coming from the data loader
# for dic in tqdm(val_loader):
#     for x in range(len(dic)):
#         z = dic[x]['image']
#         z[0] = (z[0]*0.2736+0.4662)*255
#         z[1] = (z[1]*0.2650+0.4279)*255
#         z[2] = (z[2]*0.2774+0.3946)*255
#         # print(type(z))
#         # print(z)
#         bbox = dic[x]['bbox']
#         # print(bbox)
#         boxes = []
#         boxes.append(torch.tensor([0,0,0,0]))
#         for al in range(len(bbox)):
#             boxes.append(bbox[al])
#         img=draw_bounding_boxes(z.type(torch.uint8), boxes=torch.vstack(boxes), width=4)
#         # img = 
#         show(img)
#         break
    

### Useful functions

In [27]:
iou_threshold = 0.5 # for mAP and Confusion matrix
nms_iou_threshold = 0.3

In [28]:
# def calculate_iou(gt, pr, form='pascal_voc'):
#     """Calculates the Intersection over Union.

#     Arguments:
#         gt: (torch.Tensor[N,4]) coordinates of the ground-truth boxes
#         pr: (torch.Tensor[M,4]) coordinates of the prdicted boxes
#         form: (str) gt/pred coordinates format
#             - pascal_voc: [xmin, ymin, xmax, ymax]
#             - coco: [xmin, ymin, w, h]
#     Returns:
#         iou (Tensor[N, M]): the NxM matrix containing the pairwise
#         IoU values for every element in boxes1 and boxes2
#     """
#     if form == 'coco':
#         gt = gt.clone()
#         pr = pr.clone()

#         gt[:,2] = gt[:,0] + gt[:,2]
#         gt[:,3] = gt[:,1] + gt[:,3]
#         pr[:,2] = pr[:,0] + pr[:,2]
#         pr[:,3] = pr[:,1] + pr[:,3]

#     # gt = align_coordinates(gt)
#     # pr = align_coordinates(pr)
    
#     return box_iou(gt,pr)
# iou_mat = calculate_iou(targs,preds,form='coco');
# gt_count, pr_count = iou_mat.shape
# thresh = 0.5
# iou_mat = iou_mat.where(iou_mat>thresh,tensor(0.))
# def get_mappings(iou_mat):
#     mappings = torch.zeros_like(iou_mat)
#     #first mapping (max iou for first pred_box)
#     if not iou_mat[:,0].eq(0.).all():
#         # if not a zero column
#         mappings[iou_mat[:,0].argsort()[-1],0] = 1

#     for pr_idx in range(1,pr_count):
#         # Sum of all the previous mapping columns will let 
#         # us know which gt-boxes are already assigned
#         not_assigned = torch.logical_not(mappings[:,:pr_idx].sum(1)).long()

#         # Considering unassigned gt-boxes for further evaluation 
#         targets = not_assigned * iou_mat[:,pr_idx]

#         # If no gt-box satisfy the previous conditions
#         # for the current pred-box, ignore it (False Positive)
#         if targets.eq(0).all():
#             continue

#         # max-iou from current column after all the filtering
#         # will be the pivot element for mapping
#         pivot = targets.argsort()[-1]
#         mappings[pivot,pr_idx] = 1
#     return mappings

# mappings = get_mappings(iou_mat)
# tp = mappings.sum(); tp
# fp = mappings.sum(0).eq(0).sum(); fp
# fn = mappings.sum(1).eq(0).sum(); fn
# def calculate_map(gt_boxes,pr_boxes,scores,thresh=0.5,form='pascal_voc'):
#     # sorting
#     pr_boxes = pr_boxes[scores.argsort().flip(-1)]
#     iou_mat = calculate_iou(gt_boxes,pr_boxes,form)
#     gt_count, pr_count = iou_mat.shape
    
#     # thresholding
#     iou_mat = iou_mat.where(iou_mat>thresh,tensor(0.))
    
#     mappings = get_mappings(iou_mat)
    
#     # mAP calculation
#     tp = mappings.sum()
#     fp = mappings.sum(0).eq(0).sum()
#     fn = mappings.sum(1).eq(0).sum()
#     mAP = tp / (tp+fp+fn)
    
#     return mAP

In [47]:
import torch
from collections import Counter

import torch


def intersection_over_union(boxes_preds, boxes_labels, box_format="corners"):
    """
    Calculates intersection over union
    Parameters:
        boxes_preds (tensor): Predictions of Bounding Boxes (BATCH_SIZE, 4)
        boxes_labels (tensor): Correct Labels of Boxes (BATCH_SIZE, 4)
        box_format (str): midpoint/corners, if boxes (x,y,w,h) or (x1,y1,x2,y2)
    Returns:
        tensor: Intersection over union for all examples
    """

    # Slicing idx:idx+1 in order to keep tensor dimensionality
    # Doing ... in indexing if there would be additional dimensions
    # Like for Yolo algorithm which would have (N, S, S, 4) in shape
    if box_format == "midpoint":
        box1_x1 = boxes_preds[..., 0:1] - boxes_preds[..., 2:3] / 2
        box1_y1 = boxes_preds[..., 1:2] - boxes_preds[..., 3:4] / 2
        box1_x2 = boxes_preds[..., 0:1] + boxes_preds[..., 2:3] / 2
        box1_y2 = boxes_preds[..., 1:2] + boxes_preds[..., 3:4] / 2
        box2_x1 = boxes_labels[..., 0:1] - boxes_labels[..., 2:3] / 2
        box2_y1 = boxes_labels[..., 1:2] - boxes_labels[..., 3:4] / 2
        box2_x2 = boxes_labels[..., 0:1] + boxes_labels[..., 2:3] / 2
        box2_y2 = boxes_labels[..., 1:2] + boxes_labels[..., 3:4] / 2
    elif box_format == "corners":
        box1_x1 = boxes_preds[..., 0:1]
        # print("box1_x1 : ", box1_x1)
        box1_y1 = boxes_preds[..., 1:2]
        box1_x2 = boxes_preds[..., 2:3]
        box1_y2 = boxes_preds[..., 3:4]
        box2_x1 = boxes_labels[..., 0:1]
        box2_y1 = boxes_labels[..., 1:2]
        box2_x2 = boxes_labels[..., 2:3]
        box2_y2 = boxes_labels[..., 3:4]

    # print("box format : ", box_format)
    # print(box1_x1)
    # print(box2_x1)
    x1 = torch.max(box1_x1, box2_x1)
    y1 = torch.max(box1_y1, box2_y1)
    x2 = torch.min(box1_x2, box2_x2)
    y2 = torch.min(box1_y2, box2_y2)

    # Need clamp(0) in case they do not intersect, then we want intersection to be 0
    intersection = (x2 - x1).clamp(0) * (y2 - y1).clamp(0)
    box1_area = abs((box1_x2 - box1_x1) * (box1_y2 - box1_y1))
    box2_area = abs((box2_x2 - box2_x1) * (box2_y2 - box2_y1))

    return intersection / (box1_area + box2_area - intersection + 1e-6)

def mean_average_precision(
    ls, targets_model, iou_threshold=0.5, box_format="corners", num_classes=3
):
    # print("box_format in mAP 1 : ", box_format)


    """
    Calculates mean average precision 
    Parameters:
        pred_boxes (list): list of lists containing all bboxes with each bboxes
        specified as [train_idx, class_prediction, prob_score, x1, y1, x2, y2]
        true_boxes (list): Similar as pred_boxes except all the correct ones 
        iou_threshold (float): threshold where predicted bboxes is correct
        box_format (str): "midpoint" or "corners" used to specify bboxes
        num_classes (int): number of classes
    Returns:
        float: mAP value across all classes given a specific IoU threshold 
    """
    pred_boxes = []
    for idx, ls_for_each_img in enumerate(ls):
      boxes = ls_for_each_img["boxes"]
      scores = ls_for_each_img["scores"]
      labels = ls_for_each_img["labels"]
      boxes = boxes.cpu().detach().numpy()
      scores = scores.cpu().detach().numpy()
      labels = labels.cpu().detach().numpy()
      for single_detection in zip(boxes, scores, labels):
        # print(single_detection)
        pred_boxes.append([idx, single_detection[2], single_detection[1], single_detection[0][0], single_detection[0][1], single_detection[0][2], single_detection[0][3]])
        # break
      # break
    true_boxes = []
    for idx, target_for_each_img in enumerate(targets_model):
      # print(target_for_each_img)
      boxes = target_for_each_img["boxes"]
      labels = target_for_each_img["labels"]
      boxes = boxes.cpu().detach().numpy()
      labels = labels.cpu().detach().numpy()
      for single_detection in zip(boxes, labels):
        # print(single_detection)
        true_boxes.append([idx, single_detection[1], 1, single_detection[0][0], single_detection[0][1], single_detection[0][2], single_detection[0][3]])

    # list storing all AP for respective classes
    average_precisions = []

    # used for numerical stability later on
    epsilon = 1e-6
    # print("box_format in mAP 2 : ", box_format)
    for c in range(1, num_classes+1):
        detections = []
        ground_truths = []

        # Go through all predictions and targets,
        # and only add the ones that belong to the
        # current class c
        for detection in pred_boxes:
            if detection[1] == c:
                detections.append(detection)

        for true_box in true_boxes:
            if true_box[1] == c:
                ground_truths.append(true_box)

        # find the amount of bboxes for each training example
        # Counter here finds how many ground truth bboxes we get
        # for each training example, so let's say img 0 has 3,
        # img 1 has 5 then we will obtain a dictionary with:
        # amount_bboxes = {0:3, 1:5}
        amount_bboxes = Counter([gt[0] for gt in ground_truths])

        # We then go through each key, val in this dictionary
        # and convert to the following (w.r.t same example):
        # ammount_bboxes = {0:torch.tensor[0,0,0], 1:torch.tensor[0,0,0,0,0]}
        for key, val in amount_bboxes.items():
            amount_bboxes[key] = torch.zeros(val)

        # sort by box probabilities which is index 2
        detections.sort(key=lambda x: x[2], reverse=True)
        TP = torch.zeros((len(detections)))
        FP = torch.zeros((len(detections)))
        total_true_bboxes = len(ground_truths)
        
        # If none exists for this class then we can safely skip
        if total_true_bboxes == 0:
            continue
        # print("box_format in mAP 3 : ", box_format)
        for detection_idx, detection in enumerate(detections):
            # Only take out the ground_truths that have the same
            # training idx as detection
            ground_truth_img = [
                bbox for bbox in ground_truths if bbox[0] == detection[0]
            ]

            num_gts = len(ground_truth_img)
            best_iou = 0
            # print("box_format in mAP : ", box_format)

            for idx, gt in enumerate(ground_truth_img):
                iou = intersection_over_union(
                    torch.tensor(detection[3:]),
                    torch.tensor(gt[3:]),
                    box_format=box_format,
                )

                if iou > best_iou:
                    best_iou = iou
                    best_gt_idx = idx

            if best_iou > iou_threshold:
                # only detect ground truth detection once
                if amount_bboxes[detection[0]][best_gt_idx] == 0:
                    # true positive and add this bounding box to seen
                    TP[detection_idx] = 1
                    amount_bboxes[detection[0]][best_gt_idx] = 1
                else:
                    FP[detection_idx] = 1

            # if IOU is lower then the detection is a false positive
            else:
                FP[detection_idx] = 1

        TP_cumsum = torch.cumsum(TP, dim=0)
        FP_cumsum = torch.cumsum(FP, dim=0)
        recalls = TP_cumsum / (total_true_bboxes + epsilon)
        precisions = TP_cumsum / (TP_cumsum + FP_cumsum + epsilon)
        precisions = torch.cat((torch.tensor([1]), precisions))
        recalls = torch.cat((torch.tensor([0]), recalls))
        # torch.trapz for numerical integration
        average_precisions.append(torch.trapz(precisions, recalls))

    return sum(average_precisions) / len(average_precisions), average_precisions

In [48]:
# Calculates IOU
def iou(box1, box2):
    # box1 = list(map(lambda x: int(x), box1))
    # box2 = list(map(lambda x: int(x), box2))
    a1 = (box1[2]-box1[0])*(box1[3]-box1[1])
    a2 = (box2[2]-box2[0])*(box2[3]-box2[1])
    inter = max(0, min(box1[2], box2[2]) - max(box1[0], box2[0])) * max(0, min(box1[3], box2[3]) - max(box1[1], box2[1]))
    return inter/(a1 + a2 - inter)

In [49]:
def batch_nms_confidence_filter(ls, hard = True, conf_threhold = 0.5):
  for idx in range(len(ls)):
    
    if hard: # does not give confidence filtered output
      ls_index = torchvision.ops.batched_nms(boxes = ls[idx]["boxes"], scores = ls[idx]["scores"], idxs = ls[idx]["labels"], iou_threshold = nms_iou_threshold)
      confidence_filtered_ls_index = []
      for index in ls_index:
        if ls[idx]["scores"][index] > conf_threhold:
          confidence_filtered_ls_index.append(index)
      ls_index = confidence_filtered_ls_index
    # else:    # gives confidence filtered output
    #   ls_index = soft_nms_pytorch(ls[idx]["boxes"], ls[idx]["scores"], thresh = conf_threhold) 
    
    temp_ls = {"boxes":[], "scores":[], "labels":[]}
    
    for index in ls_index:
      temp_ls["boxes"].append(ls[idx]["boxes"][index])
      temp_ls["scores"].append(ls[idx]["scores"][index])
      temp_ls["labels"].append(ls[idx]["labels"][index])
    if len(temp_ls["scores"]) > 0:
      temp_ls["boxes"] = torch.stack(temp_ls["boxes"])
      temp_ls["scores"] = torch.tensor(temp_ls["scores"])
      temp_ls["labels"] = torch.tensor(temp_ls["labels"])
    else:
      temp_ls["boxes"] = torch.tensor([[]])
      temp_ls["scores"] = torch.tensor([])
      temp_ls["labels"] = torch.tensor([])
    ls[idx] = temp_ls
  return ls

In [50]:
def confusion_matrix(ls, targets_model, TP, FP, FN, TN, iou_threshold = iou_threshold):
      for x in range(len(ls)):
        predicted_scores = ls[x]["scores"].detach().to("cpu").numpy() 
        predicted_labels = ls[x]["labels"].detach().to("cpu").numpy() 
        predicted_boxes = ls[x]["boxes"].detach().to("cpu").numpy()
        # ground_scores = targets_model[x]["scores"].detach().to("cpu").numpy() 
        ground_labels = targets_model[x]["labels"].detach().to("cpu").numpy() 
        ground_boxes =  targets_model[x]["boxes"].detach().to("cpu").numpy() 
        for index, (predicted_box, predicted_label) in enumerate(zip(predicted_boxes, predicted_labels)):
          for idx, (ground_box, ground_label) in enumerate(zip(ground_boxes, ground_labels)):
            if predicted_label == ground_label and iou(predicted_box, ground_box) > iou_threshold:
              TP += 1
              ground_labels[idx] = -1
              predicted_labels[index] = -2
              break

        not_counted = 0
        for label in ground_labels:
          if label != -1:
            not_counted += 1
        FN += not_counted

        false_counted = 0
        for label in predicted_labels:
          if label != -2:
            false_counted += 1
        FP += false_counted
      return (TP, FP, FN, TN)

## model loading

In [51]:
# my_model = torch.load("/content/drive/MyDrive/SSD detection/SSD detection_3classes_4Xdata_augmented.pth")
# model = my_model
# model.eval()

In [92]:
my_model = torch.load("/content/drive/MyDrive/SSD detection/models/SSD detection_3classes_2Xdata_augmented.pth")
model = my_model
model.eval()

SSD(
  (backbone): SSDLiteFeatureExtractorMobileNet(
    (features): Sequential(
      (0): Sequential(
        (0): ConvNormActivation(
          (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
          (1): BatchNorm2d(16, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (2): Hardswish()
        )
        (1): InvertedResidual(
          (block): Sequential(
            (0): ConvNormActivation(
              (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=16, bias=False)
              (1): BatchNorm2d(16, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
              (2): ReLU(inplace=True)
            )
            (1): ConvNormActivation(
              (0): Conv2d(16, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
              (1): BatchNorm2d(16, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
            )
          )
        )
        (2): InvertedResi

In [93]:
# vaibhav_model = torch.load("/content/vaibhav_model_dump.pth")
# model = vaibhav_model
# model.eval()

## mAP

In [94]:
temp_target = None

In [95]:
model.eval()
map_class_all = 0
map_class1 = 0
map_class2 = 0
map_class3 = 0
updates = 0
for dic in tqdm(val_loader):
    images_model = []
    targets_model = []
    for x in range(len(dic)):
        images_model.append(dic[x]['image'].float())
        dictionary = {'boxes': dic[x]['bbox'], 'labels': dic[x]['label']}
        targets_model.append(dictionary)
    ls = model.forward(images_model)
    a = ls
    b = targets_model
    ls = batch_nms_confidence_filter(ls, hard = True)
    temp_target = targets_model
    # a = ls
    # print(ls)
    # break
    # map_class1 += mean_average_precision(ls, targets_model, iou_threshold, 1,1)
    # map_class2 += mean_average_precision(ls, targets_model, iou_threshold, 2,2)
    # map_class3 += mean_average_precision(ls, targets_model, iou_threshold, 3,3)
    mAP = mean_average_precision(ls, targets_model, iou_threshold)
    map_class_all += mAP[0]
    print(mAP[1])
    # confusion_matrix
    # map = map + c
    updates += 1
# print("For class 1 map:{} | For class 2 map : {} | For class 3 map : {}".format(map_class1/updates, map_class2/updates, map_class3/updates))
print("Overall map :{}".format(map_class_all/updates))

  0%|          | 0/20 [00:00<?, ?it/s]

[tensor(1.0000), tensor(0.6375), tensor(0.5492)]
[tensor(0.8000), tensor(0.3917), tensor(0.5714)]
[tensor(0.9091), tensor(0.4000), tensor(0.6771)]
[tensor(0.8889), tensor(0.3083), tensor(0.3775)]
[tensor(0.7629), tensor(0.3333), tensor(0.5893)]
[tensor(1.0000), tensor(0.9000), tensor(0.6364)]
[tensor(0.8889), tensor(0.0556), tensor(0.6667)]
[tensor(0.7778), tensor(0.5000), tensor(0.5393)]
[tensor(0.9000), tensor(0.5000), tensor(0.5000)]
[tensor(0.8758), tensor(0.6000), tensor(0.3861)]
[tensor(0.8571), tensor(0.8333), tensor(0.4545)]
[tensor(0.6250), tensor(0.6143), tensor(0.4973)]
[tensor(0.7273), tensor(0.7500), tensor(0.7500)]
[tensor(0.7778), tensor(0.3333), tensor(0.4000)]
[tensor(0.8750), tensor(0.6133), tensor(0.4286)]
[tensor(1.0000), tensor(1.0000), tensor(1.0000)]
[tensor(0.8333), tensor(0.4854), tensor(0.4167)]
[tensor(1.0000), tensor(0.3958), tensor(0.5714)]
[tensor(0.8889), tensor(0.4000), tensor(0.9441)]
[tensor(1.0000), tensor(0.5000), tensor(0.8609)]
Overall map :0.66260

## CONFUSION MATRIX

In [96]:
model.eval()
map = 0
updates = 0
val_loader_idx = 0

TP = 0
FP = 0
FN = 0
TN = 0

for dic in tqdm(val_loader):
    images_model = []
    targets_model = []
    for x in range(len(dic)):
        images_model.append(dic[x]['image'].float())
        dictionary = {'boxes': dic[x]['bbox'], 'labels': dic[x]['label']}
        targets_model.append(dictionary)
    ls = model.forward(images_model)
    ls = batch_nms_confidence_filter(ls, hard = True)
    TP, FP, FN, TN = confusion_matrix(ls, targets_model, TP, FP, FN, TN, iou_threshold)

print("TP = {} | FP = {}".format(TP, FP))
print("-----------------")
print("FN = {} | TN = {}".format(FN, "Not defined"))

  0%|          | 0/20 [00:00<?, ?it/s]

TP = 313 | FP = 48
-----------------
FN = 129 | TN = Not defined


In [97]:
# model.eval()
# map = 0
# updates = 0
# val_loader_idx = 0
# for dic in tqdm(val_loader):
#     images_model = []
#     targets_model = []
#     for x in range(len(dic)):
#         images_model.append(dic[x]['image'].float())
#         dictionary = {'boxes': dic[x]['bbox'], 'labels': dic[x]['label']}
#         targets_model.append(dictionary)
#     ls = model.forward(images_model)
#     ls = batch_nms_confidence_filter(ls, hard = True)
#     for x in range(len(ls)):
#       z = dic[x]['image']
#       z[0] = (z[0]*0.2736+0.4662)*255
#       z[1] = (z[1]*0.2650+0.4279)*255
#       z[2] = (z[2]*0.2774+0.3946)*255
#       z = z.cpu().detach().numpy()
#       temp_img = copy.deepcopy(z)
#       temp_img = np.array(temp_img, dtype='uint8')
#       temp_img1 = np.zeros((320, 320, 3), dtype="uint8")
#       for i in range(320):
#         for j in range(320):
#           temp_img1[j][i] = (temp_img[0][j][i], temp_img[1][j][i], temp_img[2][j][i])
#       # print(temp_img)
#       temp_img1 = cv2.cvtColor(temp_img1, cv2.COLOR_BGR2RGB)
#       scores = ls[x]["scores"].detach().to("cpu").numpy() 
#       labels = ls[x]["labels"].detach().to("cpu").numpy() 
#       boxes = ls[x]["boxes"].detach().to("cpu").numpy()
#       boxes = np.array(boxes, dtype='int')
#       for label, box in zip(labels, boxes):
#         color = None
#         if label == 1:
#           color = (255, 0, 0)
#         elif label == 2:
#           color = (0, 255, 0)
#         elif label == 3:
#           color = (0, 0, 255)
#         # print(temp_img)
#         # print(box)
#         temp_img1 = cv2.rectangle(temp_img1, (box[0], box[1]), (box[2], box[3]), color, 2)
#       cv2_imshow(temp_img1)

In [98]:
a

[{'boxes': tensor([[ 74.7205,  19.5787, 172.8983, 144.3649]], grad_fn=<StackBackward0>),
  'labels': tensor([1]),
  'scores': tensor([0.9836])},
 {'boxes': tensor([[ 90.5548, 118.2727, 160.5769, 226.7522]], grad_fn=<StackBackward0>),
  'labels': tensor([1]),
  'scores': tensor([0.9990])},
 {'boxes': tensor([[129.4462, 123.8077, 248.7479, 273.1547]], grad_fn=<StackBackward0>),
  'labels': tensor([1]),
  'scores': tensor([0.9949])},
 {'boxes': tensor([[190.4005, 211.5965, 271.9470, 307.7177]], grad_fn=<StackBackward0>),
  'labels': tensor([1]),
  'scores': tensor([0.9932])},
 {'boxes': tensor([[139.2632,  67.3201, 236.9120, 202.3436]], grad_fn=<StackBackward0>),
  'labels': tensor([1]),
  'scores': tensor([0.9980])},
 {'boxes': tensor([[ 68.3663,  30.1075, 170.2031, 160.7437]], grad_fn=<StackBackward0>),
  'labels': tensor([1]),
  'scores': tensor([0.9531])},
 {'boxes': tensor([[255.0121,  86.8395, 286.9255, 196.0137],
          [143.9222,  65.3582, 195.3079, 175.6862]], grad_fn=<StackBa

In [99]:
b = a[-3:]
print(b)

[{'boxes': tensor([[144.1634,  49.2664, 259.5536, 271.4836],
        [ 46.4425,   4.6094, 156.1039, 311.0587],
        [132.6616, 100.5494, 174.3637, 236.4232]], grad_fn=<StackBackward0>), 'scores': tensor([0.9973, 0.9773, 0.5589]), 'labels': tensor([3, 3, 3])}, {'boxes': tensor([[ 70.8878,  64.3830, 230.2723, 294.9669],
        [263.4974, 110.6995, 297.5823, 175.6465]], grad_fn=<StackBackward0>), 'scores': tensor([0.9581, 0.8994]), 'labels': tensor([2, 3])}, {'boxes': tensor([[ 95.7570, 131.6015, 182.8741, 261.0623],
        [ 57.4863, 121.2908, 173.3777, 252.7001]], grad_fn=<StackBackward0>), 'scores': tensor([0.9793, 0.8479]), 'labels': tensor([1, 3])}]


In [100]:
b

[{'boxes': tensor([[144.1634,  49.2664, 259.5536, 271.4836],
          [ 46.4425,   4.6094, 156.1039, 311.0587],
          [132.6616, 100.5494, 174.3637, 236.4232]], grad_fn=<StackBackward0>),
  'labels': tensor([3, 3, 3]),
  'scores': tensor([0.9973, 0.9773, 0.5589])},
 {'boxes': tensor([[ 70.8878,  64.3830, 230.2723, 294.9669],
          [263.4974, 110.6995, 297.5823, 175.6465]], grad_fn=<StackBackward0>),
  'labels': tensor([2, 3]),
  'scores': tensor([0.9581, 0.8994])},
 {'boxes': tensor([[ 95.7570, 131.6015, 182.8741, 261.0623],
          [ 57.4863, 121.2908, 173.3777, 252.7001]], grad_fn=<StackBackward0>),
  'labels': tensor([1, 3]),
  'scores': tensor([0.9793, 0.8479])}]

In [101]:
preds = []
for idx, ls_for_each_img in enumerate(b):
  # print(idx, ls_for_each_img)
  boxes = ls_for_each_img["boxes"]
  scores = ls_for_each_img["scores"]
  labels = ls_for_each_img["labels"]
  boxes = boxes.cpu().detach().numpy()
  scores = scores.cpu().detach().numpy()
  labels = labels.cpu().detach().numpy()
  for single_detection in zip(boxes, scores, labels):
    print(single_detection)
    # print(np.shape(single_detection[1]))
    # if np.shape(single_detection[1])[1] == 0:
    #   continue
    preds.append([idx, single_detection[2], single_detection[1], single_detection[0][0], single_detection[0][1], single_detection[0][2], single_detection[0][3]])

print(preds)

(array([144.16339,  49.26644, 259.55356, 271.4836 ], dtype=float32), 0.99726784, 3)
(array([ 46.44245  ,   4.6094055, 156.10388  , 311.05875  ], dtype=float32), 0.97725004, 3)
(array([132.66158, 100.5494 , 174.36372, 236.42319], dtype=float32), 0.55890584, 3)
(array([ 70.887825,  64.38302 , 230.27231 , 294.96695 ], dtype=float32), 0.9580607, 2)
(array([263.49738, 110.69947, 297.58234, 175.64648], dtype=float32), 0.8993826, 3)
(array([ 95.75699, 131.60147, 182.87408, 261.06232], dtype=float32), 0.9792961, 1)
(array([ 57.48625 , 121.290794, 173.3777  , 252.70007 ], dtype=float32), 0.8478921, 3)
[[0, 3, 0.99726784, 144.16339, 49.26644, 259.55356, 271.4836], [0, 3, 0.97725004, 46.44245, 4.6094055, 156.10388, 311.05875], [0, 3, 0.55890584, 132.66158, 100.5494, 174.36372, 236.42319], [1, 2, 0.9580607, 70.887825, 64.38302, 230.27231, 294.96695], [1, 3, 0.8993826, 263.49738, 110.69947, 297.58234, 175.64648], [2, 1, 0.9792961, 95.75699, 131.60147, 182.87408, 261.06232], [2, 3, 0.8478921, 57.486

In [102]:
temp_target[0]

{'boxes': tensor([[ 75,  18, 171, 148]], dtype=torch.int32),
 'labels': tensor([1])}

In [103]:
true_boxes = []
for idx, target_for_each_img in enumerate(temp_target):
  # print(target_for_each_img)
  boxes = target_for_each_img["boxes"]
  labels = target_for_each_img["labels"]
  boxes = boxes.cpu().detach().numpy()
  labels = labels.cpu().detach().numpy()
  for single_detection in zip(boxes, labels):
    # print(single_detection)
    true_boxes.append([idx, single_detection[1], 1, single_detection[0][0], single_detection[0][1], single_detection[0][2], single_detection[0][3]])
print(true_boxes)

[[0, 1, 1, 75, 18, 171, 148], [1, 1, 1, 91, 118, 165, 216], [2, 1, 1, 127, 121, 248, 270], [3, 1, 1, 194, 211, 276, 305], [4, 1, 1, 140, 74, 237, 205], [5, 1, 1, 72, 26, 166, 152], [6, 2, 1, 162, 140, 176, 181], [6, 3, 1, 256, 84, 291, 196], [6, 3, 1, 147, 49, 198, 152], [6, 3, 1, 136, 136, 157, 172], [7, 2, 1, 91, 85, 161, 278], [7, 3, 1, 164, 58, 221, 273], [8, 1, 1, 16, 1, 217, 262], [9, 3, 1, 0, 246, 62, 317], [10, 1, 1, 39, 69, 202, 300], [11, 2, 1, 128, 151, 263, 316], [12, 2, 1, 139, 167, 161, 223], [13, 2, 1, 164, 132, 216, 251], [13, 3, 1, 156, 58, 250, 268], [13, 3, 1, 43, 0, 145, 312], [13, 3, 1, 134, 85, 169, 231], [14, 2, 1, 63, 61, 246, 296], [14, 3, 1, 263, 110, 298, 177], [15, 1, 1, 88, 126, 185, 275]]


In [104]:
temp_trash = ([165.23242, 146.69205, 291.3508 , 304.7331 ], 0.9991553, 1)
print(temp_trash)

([165.23242, 146.69205, 291.3508, 304.7331], 0.9991553, 1)
